In [3]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup

In [32]:
def fetch_product_details(page_number, endpoint, count=12):
    url = f'https://cierraperfumes.com/product-category/{endpoint}/page/{page_number}/?count={count}'
    response = requests.get(url)
    
    # Verify successful response
    if response.status_code != 200:
        print(f"Failed to retrieve page {page_number}")
        return None

    # Parse HTML with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    products = soup.find_all('div', class_='product-inner')  # Locate each product container
    
    # List to hold product details
    product_list = []
    
    # Extract details for each product
    for product in products:
        category_text = url.split('/')[-4].title()
        
        # Get product title
        title = product.find('h2', class_='woocommerce-loop-product__title')
        title_text = title.get_text(strip=True) if title else None
        
        # Get price
        price = product.find('span', class_='woocommerce-Price-amount amount')
        price_text = price.get_text(strip=True) if price else None
        
        # Get product link
        link_tag = product.find('a', class_='tf-loop-product-thumbs-link')
        product_link = link_tag['href'] if link_tag else None
        
        # Get thumbnail image link
        image_tag = link_tag.find('img', class_='attachment-woocommerce_thumbnail size-woocommerce_thumbnail') if link_tag else None
        image_link = image_tag['src'] if image_tag else None
        
        # Add product details to the list
        product_list.append({
            "category": category_text,
            "title": title_text,
            "price": price_text,
            "link": product_link,
            "image": image_link
        })
    
    # Convert to JSON
    return product_list


def fetch_all_pages(endpoint, count=12):
    page_number = 1
    all_products = []

    # Loop over pages until an error occurs
    while True:
        print(f"Fetching page {page_number}...")
        product_data = fetch_product_details(page_number, endpoint, count)
        
        # Stop if there are no products found
        if not product_data:
            break
        
        # Append products to the master list
        all_products.extend(product_data)
        page_number += 1

    # Convert to JSON for output
    return all_products


def fetch_more_info(link):
    response = requests.get(link)
    
    # Check for a successful response
    if response.status_code != 200:
        print(f"Failed to retrieve data from {link}")
        return None
    
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extract description and notes
    description_tag = soup.select_one('#tab-description p:nth-of-type(1)')
    top_tag = soup.select_one('#tab-description p:nth-of-type(2)')
    middle_tag = soup.select_one('#tab-description p:nth-of-type(3)')
    base_tag = soup.select_one('#tab-description p:nth-of-type(4)')
    
    # Parse and clean the content
    description = description_tag.get_text(strip=True) if description_tag else None
    top = top_tag.get_text(strip=True).replace("Top – ", "") if top_tag else None
    middle = middle_tag.get_text(strip=True).replace("Middle – ", "") if middle_tag else None
    base = base_tag.get_text(strip=True).replace("Base – ", "") if base_tag else None
    
    # Return data as a dictionary
    return {
        "description": description,
        "top": top,
        "middle": middle,
        "base": base
    }

In [18]:
endpoint = 'men'
men_perfumes = fetch_all_pages(endpoint)

Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...
Fetching page 5...
Fetching page 6...
Fetching page 7...
Fetching page 8...
Fetching page 9...
Fetching page 10...
Fetching page 11...
Fetching page 12...
Fetching page 13...
Fetching page 14...
Fetching page 15...
Fetching page 16...
Fetching page 17...
Fetching page 18...
Fetching page 19...
Fetching page 20...
Fetching page 21...
Fetching page 22...
Fetching page 23...
Fetching page 24...
Fetching page 25...
Fetching page 26...
Fetching page 27...
Fetching page 28...
Fetching page 29...
Fetching page 30...
Fetching page 31...
Fetching page 32...
Fetching page 33...
Fetching page 34...
Fetching page 35...
Fetching page 36...
Fetching page 37...
Fetching page 38...
Fetching page 39...
Fetching page 40...
Fetching page 41...
Fetching page 42...
Fetching page 43...
Fetching page 44...
Fetching page 45...
Fetching page 46...
Fetching page 47...
Fetching page 48...
Fetching page 49...
Fetching page 50...
Fetching 

In [20]:
men_df = pd.DataFrame(men_perfumes)
men_df

,category,title,price,link,image
0,Men,"Aberchrombie & Fitch, Authentic Edt 100Ml M","KSh7,300.00",https://cierraperfumes.com/product/aberchrombi...,https://i0.wp.com/cierraperfumes.com/wp-conten...
1,Men,"Aberchrombie & Fitch, Authentic Night Edt 100Ml M","KSh7,300.00",https://cierraperfumes.com/product/aberchrombi...,https://i0.wp.com/cierraperfumes.com/wp-conten...
2,Men,"Aberchrombie & Fitch, Fierce Edc 100Ml M","KSh15,500.00",https://cierraperfumes.com/product/aberchrombi...,https://i0.wp.com/cierraperfumes.com/wp-conten...
3,Men,"ABERCHROMBIE & FITCH, FIRST INSTINCT EDT 100ML...","KSh8,500.00",https://cierraperfumes.com/product/aberchrombi...,https://i0.wp.com/cierraperfumes.com/wp-conten...
4,Men,"Acqua Di Parma, Colonia Club Edc 100Ml M","KSh18,400.00",https://cierraperfumes.com/product/acqua-di-pa...,https://i0.wp.com/cierraperfumes.com/wp-conten...
...,...,...,...,...,...
683,Men,"Ysl, Y Edp 100Ml M Gift Set","KSh20,500.00",https://cierraperfumes.com/product/ysl-y-edp-1...,https://i0.wp.com/cierraperfumes.com/wp-conten...
684,Men,"Ysl, Y Edt 100Ml M","KSh15,700.00",https://cierraperfumes.com/product/ysl-y-edt-1...,https://i0.wp.com/cierraperfumes.com/wp-conten...
685,Men,"YSL, Y INTENSE EDP 100ML M","KSh20,900.00",https://cierraperfumes.com/product/ysl-y-inten...,https://i0.wp.com/cierraperfumes.com/wp-conten...
686,Men,"Ysl, Y Le Parfum 100Ml M","KSh22,000.00",https://cierraperfumes.com/product/ysl-y-le-pa...,https://i0.wp.com/cierraperfumes.com/wp-conten...


In [21]:
endpoint = 'women'
women_perfumes = fetch_all_pages(endpoint, count=60)

Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...
Fetching page 5...
Fetching page 6...
Fetching page 7...
Fetching page 8...
Fetching page 9...
Fetching page 10...
Fetching page 11...
Fetching page 12...
Fetching page 13...
Fetching page 14...
Fetching page 15...
Fetching page 16...
Fetching page 17...
Fetching page 18...
Failed to retrieve page 18


In [22]:
women_df = pd.DataFrame(women_perfumes)
women_df

,category,title,price,link,image
0,Women,"Aberchrombie & Fitch, Authentic Away Edp 100Ml...","KSh10,900.00",https://cierraperfumes.com/product/aberchrombi...,https://i0.wp.com/cierraperfumes.com/wp-conten...
1,Women,"Aberchrombie & Fitch, Authentic Night Edp 100Ml W","KSh7,400.00",https://cierraperfumes.com/product/aberchrombi...,https://i0.wp.com/cierraperfumes.com/wp-conten...
2,Women,"Aberchrombie & Fitch, Authentic Night Edp 100M...","KSh9,100.00",https://cierraperfumes.com/product/aberchrombi...,https://i0.wp.com/cierraperfumes.com/wp-conten...
3,Women,"AFNAN, 9AM EDP 100ML W","KSh6,500.00",https://cierraperfumes.com/product/afnan-9am-e...,https://i0.wp.com/cierraperfumes.com/wp-conten...
4,Women,"AFNAN, VIOLET BOUQUET EDP 100ML W","KSh7,300.00",https://cierraperfumes.com/product/afnan-viole...,https://i0.wp.com/cierraperfumes.com/wp-conten...
...,...,...,...,...,...
995,Women,"Ysl, Supreme Bouquet Edp 75Ml W","KSh43,300.00",https://cierraperfumes.com/product/ysl-supreme...,https://i0.wp.com/cierraperfumes.com/wp-conten...
996,Women,"Zadig & Voltaire, This Is Her Edp 100Ml W","KSh12,400.00",https://cierraperfumes.com/product/zadig-volta...,https://i0.wp.com/cierraperfumes.com/wp-conten...
997,Women,"Zadig & Voltaire, This Is Her Undressed Edp 10...","KSh16,200.00",https://cierraperfumes.com/product/zadig-volta...,https://i0.wp.com/cierraperfumes.com/wp-conten...
998,Women,"Zadig & Voltaire, This Is Her Vibes Of Freedom...","KSh14,500.00",https://cierraperfumes.com/product/zadig-volta...,https://i0.wp.com/cierraperfumes.com/wp-conten...


In [23]:
endpoint = 'unisex'
unisex_perfumes = fetch_all_pages(endpoint, count=60)

Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...
Fetching page 5...
Fetching page 6...
Fetching page 7...
Fetching page 8...
Fetching page 9...
Fetching page 10...
Fetching page 11...
Fetching page 12...
Fetching page 13...
Fetching page 14...
Fetching page 15...
Failed to retrieve page 15


In [24]:
unisex_df = pd.DataFrame(unisex_perfumes)
unisex_df

,category,title,price,link,image
0,Unisex,"A Lab Of Fire, Eau D’ Ipanema Edp 60Ml U","KSh15,300.00",https://cierraperfumes.com/product/a-lab-of-fi...,https://i0.wp.com/cierraperfumes.com/wp-conten...
1,Unisex,"Accendis, Lucevera Edp 100Ml U","KSh15,100.00",https://cierraperfumes.com/product/accendis-lu...,https://i0.wp.com/cierraperfumes.com/wp-conten...
2,Unisex,"Acqua Di Parma, Colonia Club Edc 180Ml U","KSh19,700.00",https://cierraperfumes.com/product/acqua-di-pa...,https://i0.wp.com/cierraperfumes.com/wp-conten...
3,Unisex,"Acqua Di Parma, Colonia Edc 100Ml U Gift Set","KSh15,300.00",https://cierraperfumes.com/product/acqua-di-pa...,https://i0.wp.com/cierraperfumes.com/wp-conten...
4,Unisex,"Acqua Di Parma, Colonia Futura Edc 100Ml U Gif...","KSh15,100.00",https://cierraperfumes.com/product/acqua-di-pa...,https://i0.wp.com/cierraperfumes.com/wp-conten...
...,...,...,...,...,...
786,Unisex,"YSL, ROUGE VELOURS EDP 125ML U","KSh50,700.00",https://cierraperfumes.com/product/ysl-rouge-v...,https://i0.wp.com/cierraperfumes.com/wp-conten...
787,Unisex,"Ysl, Saharienne Neroli Musc Blancs Edp 125Ml U","KSh48,800.00",https://cierraperfumes.com/product/ysl-saharie...,https://i0.wp.com/cierraperfumes.com/wp-conten...
788,Unisex,"Ysl, Sleek Suede Edp 75Ml U","KSh28,700.00",https://cierraperfumes.com/product/ysl-sleek-s...,https://i0.wp.com/cierraperfumes.com/wp-conten...
789,Unisex,"Ysl, Splendid Wood Edp 80Ml U","KSh28,700.00",https://cierraperfumes.com/product/ysl-splendi...,https://i0.wp.com/cierraperfumes.com/wp-conten...


In [25]:
cierra_df = pd.concat([women_df, men_df, unisex_df], ignore_index=True)

In [26]:
cierra_df.to_csv('data/cierra.csv', index=False, encoding='utf-8')

In [28]:
cierra_df['link']

0       https://cierraperfumes.com/product/aberchrombi...
1       https://cierraperfumes.com/product/aberchrombi...
2       https://cierraperfumes.com/product/aberchrombi...
3       https://cierraperfumes.com/product/afnan-9am-e...
4       https://cierraperfumes.com/product/afnan-viole...
                              ...                        
2474    https://cierraperfumes.com/product/ysl-rouge-v...
2475    https://cierraperfumes.com/product/ysl-saharie...
2476    https://cierraperfumes.com/product/ysl-sleek-s...
2477    https://cierraperfumes.com/product/ysl-splendi...
2478    https://cierraperfumes.com/product/ysl-trench-...
Name: link, Length: 2479, dtype: object

In [30]:
fetch_more_info('https://cierraperfumes.com/product/aberchrombie-fitch-authentic-edt-100ml-m/')

{'description': 'Authentic for Men is an aromatic fragrance with refreshing citrus overtone. The lightness of the scent is balanced with warm, earthy and masculine elements. The sharp spice of ginger, the savory nature of sage and calming lavender come together to form the heart of the scent. Earthy clearwood and the slight bitterness of moss lend a touch of masculinity to the base.',
 'top': 'Grapefruit, Bergamont, Pink Pepper.',
 'middle': 'Ginger, Sage, Lavender.',
 'base': 'Clearwood, Moss, Suede.'}

In [31]:
cierra_df

,category,title,price,link,image
0,Women,"Aberchrombie & Fitch, Authentic Away Edp 100Ml...","KSh10,900.00",https://cierraperfumes.com/product/aberchrombi...,https://i0.wp.com/cierraperfumes.com/wp-conten...
1,Women,"Aberchrombie & Fitch, Authentic Night Edp 100Ml W","KSh7,400.00",https://cierraperfumes.com/product/aberchrombi...,https://i0.wp.com/cierraperfumes.com/wp-conten...
2,Women,"Aberchrombie & Fitch, Authentic Night Edp 100M...","KSh9,100.00",https://cierraperfumes.com/product/aberchrombi...,https://i0.wp.com/cierraperfumes.com/wp-conten...
3,Women,"AFNAN, 9AM EDP 100ML W","KSh6,500.00",https://cierraperfumes.com/product/afnan-9am-e...,https://i0.wp.com/cierraperfumes.com/wp-conten...
4,Women,"AFNAN, VIOLET BOUQUET EDP 100ML W","KSh7,300.00",https://cierraperfumes.com/product/afnan-viole...,https://i0.wp.com/cierraperfumes.com/wp-conten...
...,...,...,...,...,...
2474,Unisex,"YSL, ROUGE VELOURS EDP 125ML U","KSh50,700.00",https://cierraperfumes.com/product/ysl-rouge-v...,https://i0.wp.com/cierraperfumes.com/wp-conten...
2475,Unisex,"Ysl, Saharienne Neroli Musc Blancs Edp 125Ml U","KSh48,800.00",https://cierraperfumes.com/product/ysl-saharie...,https://i0.wp.com/cierraperfumes.com/wp-conten...
2476,Unisex,"Ysl, Sleek Suede Edp 75Ml U","KSh28,700.00",https://cierraperfumes.com/product/ysl-sleek-s...,https://i0.wp.com/cierraperfumes.com/wp-conten...
2477,Unisex,"Ysl, Splendid Wood Edp 80Ml U","KSh28,700.00",https://cierraperfumes.com/product/ysl-splendi...,https://i0.wp.com/cierraperfumes.com/wp-conten...


In [33]:
details_df = cierra_df['link'].apply(fetch_more_info).apply(pd.Series)
details_df

,description,top,middle,base
0,"The journey is a trip, but the destination is ...","Top Notes: Brazilian Mandarin, Red Currant, Pear","Heart Notes: Plush Pink Freesia, Muguet and Ja...","Base Notes: Warm Praline, Amber, Musk"
1,Authentic Night for Women is for the woman who...,Fragrance Notes:,"A mouth-watering combination of Black Currant,...",None
2,Authentic Night by Abercrombie & Fitch 100ml E...,3 Piece Set Includes:,– 100ml Eau De Parfum Spray– 200ml Perfumed Bo...,Authentic Night for Women is for the woman who...
3,Afnan 9am perfume for women is a fruity-spicy ...,"Top Notes: Mandarin, Cedrat, Cardamom and Pink...","Middle Notes: Lavender, Orange Blossom, Rose a...","Base Notes: Cedarwood, Moss, Patchouli and Musk"
4,Violet Bouquet is an opulent signature scent o...,"Top notes: Sweet, Woody, Saffron, Citrus.","Middle notes: Aromatic, Amber, Orange, Citrus ...","Base notes: Fresh Spicy, Ambergris, Oak Moss, ..."
...,...,...,...,...
2474,For the passionate and audacious. Velvety patc...,From the sumptuous jet-embroidered bolero of t...,"Top notes: white tea accord, pink pepper","Heart notes: rose, jasmine, orris"
2475,With its cotton fabric and signature sandy hue...,None,None,None
2476,Inspired by Mr Yves Saint Laurent’s retreats i...,"Sleek Suede perfume is a captivating scent, wi...",None,None
2477,Inspired by Mr Yves Saint Laurent’s retreats i...,None,None,None


In [34]:
cierra_df = pd.concat([cierra_df, details_df], axis=1)
cierra_df

,category,title,price,link,image,description,top,middle,base
0,Women,"Aberchrombie & Fitch, Authentic Away Edp 100Ml...","KSh10,900.00",https://cierraperfumes.com/product/aberchrombi...,https://i0.wp.com/cierraperfumes.com/wp-conten...,"The journey is a trip, but the destination is ...","Top Notes: Brazilian Mandarin, Red Currant, Pear","Heart Notes: Plush Pink Freesia, Muguet and Ja...","Base Notes: Warm Praline, Amber, Musk"
1,Women,"Aberchrombie & Fitch, Authentic Night Edp 100Ml W","KSh7,400.00",https://cierraperfumes.com/product/aberchrombi...,https://i0.wp.com/cierraperfumes.com/wp-conten...,Authentic Night for Women is for the woman who...,Fragrance Notes:,"A mouth-watering combination of Black Currant,...",None
2,Women,"Aberchrombie & Fitch, Authentic Night Edp 100M...","KSh9,100.00",https://cierraperfumes.com/product/aberchrombi...,https://i0.wp.com/cierraperfumes.com/wp-conten...,Authentic Night by Abercrombie & Fitch 100ml E...,3 Piece Set Includes:,– 100ml Eau De Parfum Spray– 200ml Perfumed Bo...,Authentic Night for Women is for the woman who...
3,Women,"AFNAN, 9AM EDP 100ML W","KSh6,500.00",https://cierraperfumes.com/product/afnan-9am-e...,https://i0.wp.com/cierraperfumes.com/wp-conten...,Afnan 9am perfume for women is a fruity-spicy ...,"Top Notes: Mandarin, Cedrat, Cardamom and Pink...","Middle Notes: Lavender, Orange Blossom, Rose a...","Base Notes: Cedarwood, Moss, Patchouli and Musk"
4,Women,"AFNAN, VIOLET BOUQUET EDP 100ML W","KSh7,300.00",https://cierraperfumes.com/product/afnan-viole...,https://i0.wp.com/cierraperfumes.com/wp-conten...,Violet Bouquet is an opulent signature scent o...,"Top notes: Sweet, Woody, Saffron, Citrus.","Middle notes: Aromatic, Amber, Orange, Citrus ...","Base notes: Fresh Spicy, Ambergris, Oak Moss, ..."
...,...,...,...,...,...,...,...,...,...
2474,Unisex,"YSL, ROUGE VELOURS EDP 125ML U","KSh50,700.00",https://cierraperfumes.com/product/ysl-rouge-v...,https://i0.wp.com/cierraperfumes.com/wp-conten...,For the passionate and audacious. Velvety patc...,From the sumptuous jet-embroidered bolero of t...,"Top notes: white tea accord, pink pepper","Heart notes: rose, jasmine, orris"
2475,Unisex,"Ysl, Saharienne Neroli Musc Blancs Edp 125Ml U","KSh48,800.00",https://cierraperfumes.com/product/ysl-saharie...,https://i0.wp.com/cierraperfumes.com/wp-conten...,With its cotton fabric and signature sandy hue...,None,None,None
2476,Unisex,"Ysl, Sleek Suede Edp 75Ml U","KSh28,700.00",https://cierraperfumes.com/product/ysl-sleek-s...,https://i0.wp.com/cierraperfumes.com/wp-conten...,Inspired by Mr Yves Saint Laurent’s retreats i...,"Sleek Suede perfume is a captivating scent, wi...",None,None
2477,Unisex,"Ysl, Splendid Wood Edp 80Ml U","KSh28,700.00",https://cierraperfumes.com/product/ysl-splendi...,https://i0.wp.com/cierraperfumes.com/wp-conten...,Inspired by Mr Yves Saint Laurent’s retreats i...,None,None,None


In [35]:
cierra_df.to_csv('data/cierra_updated.csv', index=False, encoding='utf-8')